In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [3]:
classifier = Sequential()

# Convolution Step 1
classifier.add(Convolution2D(96, 11, strides = (4, 4), padding = 'valid', input_shape=(224, 224, 3), activation = 'relu'))

# Max Pooling Step 1
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Convolution Step 2
classifier.add(Convolution2D(256, 11, strides = (1, 1), padding='valid', activation = 'relu'))

# Max Pooling Step 2
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='valid'))
classifier.add(BatchNormalization())

# Convolution Step 3
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 4
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 5
classifier.add(Convolution2D(256, 3, strides=(1,1), padding='valid', activation = 'relu'))

# Max Pooling Step 3
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Flattening Step
classifier.add(Flatten())

In [4]:

classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 1000, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 10, activation = 'softmax'))
classifier.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 256)         1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 384)         8

In [5]:
classifier.compile(optimizer=optimizers.SGD(lr=0.001, momentum=0.9, decay=0.005),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])



# image preprocessing
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=40,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

In [6]:

train_data_dir = "/content/drive/My Drive/train_images"     # directory of training data

test_data_dir = "/content/drive/My Drive/TestSet"

In [7]:
training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(224, 224),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory(test_data_dir,
                                            target_size=(224, 224),
                                            batch_size=batch_size,
                                            class_mode='categorical')

print(training_set.class_indices)


Found 62 images belonging to 10 classes.
Found 0 images belonging to 0 classes.
{'APHID': 0, 'BACTERIA': 1, 'BSIGATOKA': 2, 'HEALTHY': 3, 'MOSAIC': 4, 'MOSAICVIRUS': 5, 'PANAMAWILT': 6, 'PWILT': 7, 'STREAK': 8, 'YSIGATOKA': 9}


In [8]:
history = classifier.fit_generator(training_set,
                                   steps_per_epoch=training_set.samples//batch_size,
                                   validation_data=test_set,
                                   epochs=50,
                                   validation_steps=test_set.samples//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
1/1 [==============================] - 0s 1ms/step - loss: 3.0580 - accuracy: 0.0333
Epoch 2/50
1/1 [==============================] - 0s 1ms/step - loss: 3.2516 - accuracy: 0.1333
Epoch 3/50
1/1 [==============================] - 0s 1ms/step - loss: 2.8830 - accuracy: 0.1333
Epoch 4/50
1/1 [==============================] - 0s 1ms/step - loss: 3.1895 - accuracy: 0.0667
Epoch 5/50
1/1 [==============================] - 0s 1ms/step - loss: 2.8697 - accuracy: 0.0938
Epoch 6/50
1/1 [==============================] - 0s 2ms/step - loss: 2.6981 - accuracy: 0.2188
Epoch 7/50
1/1 [==============================] - 0s 1ms/step - loss: 2.7164 - accuracy: 0.2667
Epoch 8/50
1/1 [==============================] - 0s 1ms/step - loss: 2.2907 - accuracy: 0.1667
Epoch 9/50
1/1 [==============================] - 0s 1ms/step - loss: 2.4564 - accuracy: 0.0938
Epoch 10/50
1/1 [==============================] - 0s 1ms/st

In [9]:
filepath="model2.hdf5"
classifier.save(filepath)

In [45]:
import tensorflow as tf
MODEL_PATH = "/content/model2.hdf5"

IMG_PATH = "/content/drive/My Drive/TuneSet/STREAK/STREAK_images_004.jpg"
model = tf.keras.models.load_model(MODEL_PATH)

In [46]:
import numpy as np
from tensorflow.keras.preprocessing import image
img = image.load_img(IMG_PATH, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255

In [47]:

prediction = model.predict(img)

In [49]:
prediction_flatten = prediction.flatten()
max_val_index = np.argmax(prediction_flatten)

In [50]:

output_dict ={'APHID' :0,
              'BACTERIA':1,
              "BSIGATOKA":2,
              "HEALTHY":3,
              "MOSAIC":4,
              "MOSAICVIRUS":5,
              "PANAMAWILT":6,
              "PWILT":7,
              "STREAK":8,
              "YSIGATOKA":9

              
    
}
output_list = list(output_dict.keys())

In [51]:
result = output_list[max_val_index]

In [52]:
print(result)

HEALTHY
